In [ ]:
# Cellpose segmentation for Stereo-seq Mouse Brain data
import os
import numpy as np
import tifffile
import matplotlib.pyplot as plt
from cellpose import models, core, io, plot
from pathlib import Path
from tqdm import tqdm
from natsort import natsorted
import time

# Set up logging
io.logger_setup()

# Stereo-seq image path - Mouse Brain Adult
dapi_path = "D:/paper/newmodel/data/stereo_seq_mouse_brain/Mouse_brain_Adult_sub.tif"

# Output directory for segmentation results
output_dir = "cellpose_segmentation"
os.makedirs(output_dir, exist_ok=True)

# Load Stereo-seq image
print(f"Loading Stereo-seq image: {dapi_path}")
if not os.path.exists(dapi_path):
    print(f"Error: Stereo-seq image file not found: {dapi_path}")
    exit()

dapi_img = tifffile.imread(dapi_path)
print(f"Original image shape: {dapi_img.shape}")
print(f"Original image dtype: {dapi_img.dtype}, max: {dapi_img.max()}, min: {dapi_img.min()}")

# Image preprocessing - handle possible multi-dimensional images
if len(dapi_img.shape) == 3:
    # If 3D image (possibly Z, Y, X or Y, X, C)
    if dapi_img.shape[0] < dapi_img.shape[1]:  # Channel might be in the first dimension
        # Assume (C, H, W) format, convert to (H, W, C)
        dapi_img = np.transpose(dapi_img, (1, 2, 0))
        print(f"Converted image format to (H, W, C): {dapi_img.shape}")
    # If multiple Z layers, take maximum projection or middle layer
    if dapi_img.shape[-1] > 3:  # If too many channels, might be Z layers
        print("Detected multi-layer image, using maximum intensity projection")
        dapi_img = np.max(dapi_img, axis=-1, keepdims=True)
elif len(dapi_img.shape) == 2:
    # If 2D image, convert to 3D (H, W, C)
    dapi_img = np.expand_dims(dapi_img, axis=-1)
    print(f"Converted image shape: {dapi_img.shape}")

# Image normalization
if dapi_img.max() > 0:
    if dapi_img.dtype == np.uint16:
        print("Image is uint16 type, keeping original format")
        dapi_img_norm = dapi_img.copy()
    elif dapi_img.max() > 255:
        print("Normalizing image to 0-255 (uint8)")
        dapi_img_norm = (dapi_img / dapi_img.max() * 255).astype(np.uint8)
    else:
        print("Image is already in suitable intensity range")
        dapi_img_norm = dapi_img.astype(np.uint8)
else:
    print("Warning: Image max value is 0, might be empty image")
    dapi_img_norm = dapi_img.astype(np.uint8)

# Channel selection - according to Cellpose format
# For Stereo-seq single-channel image, we use the first channel
first_channel = '0'   # Use the first channel
second_channel = 'None'  # No second channel
third_channel = 'None'   # No third channel

selected_channels = []
for i, c in enumerate([first_channel, second_channel, third_channel]):
    if c == 'None':
        continue
    if int(c) >= dapi_img_norm.shape[-1]:  # Fixed boundary check
        print(f"Warning: Channel index {c} exceeds image channel count {dapi_img_norm.shape[-1]}")
        continue
    if c != 'None':
        selected_channels.append(int(c))

print(f"Selected channels: {selected_channels}")

# Create image with selected channels
img_selected_channels = np.zeros_like(dapi_img_norm)
if len(selected_channels) > 0:
    img_selected_channels[:, :, :len(selected_channels)] = dapi_img_norm[:, :, selected_channels]
else:
    # If no channels selected, use original image
    img_selected_channels = dapi_img_norm

print(f"Processed image shape: {img_selected_channels.shape}")

# Load the Cellpose model with GPU enabled
print("Initializing Cellpose model (cyto3)...")
model = models.Cellpose(gpu=True, model_type='cyto3')

# Start timing
start_time = time.time()

# Perform segmentation
print("Running Cellpose segmentation...")
masks, flows, styles, diams = model.eval(img_selected_channels, diameter=None, channels=[0, 0])

# End timing
end_time = time.time()

# Calculate elapsed time
elapsed_time = end_time - start_time

# Count the number of cells in the mask
unique_cells = np.unique(masks)
cell_count = len(unique_cells) - 1  # Subtract 1 to exclude the background (value 0)

# Print results
print(f"Processed {dapi_path} in {elapsed_time:.2f} seconds using GPU.")
print(f"Segmentation complete. Found {cell_count} cells.")

# Save segmentation results
mask_filename = "cellpose_masks_stereo_seq_mouse_brain.tif"
mask_path = os.path.join(output_dir, mask_filename)
tifffile.imwrite(mask_path, masks.astype(np.uint32))
print(f"Mask file saved: {mask_path}")


2025-11-07 11:40:55,415 [INFO] WRITING LOG OUTPUT TO C:\Users\lyh\.cellpose\run.log
2025-11-07 11:40:55,416 [INFO] 
cellpose version: 	3.0.7 
platform:       	win32 
python version: 	3.9.21 
torch version:  	2.0.1+cu118
2025-11-07 11:40:55,416 [INFO] 
cellpose version: 	3.0.7 
platform:       	win32 
python version: 	3.9.21 
torch version:  	2.0.1+cu118
Loading Stereo-seq image: D:/paper/newmodel/data/stereo_seq_mouse_brain/Mouse_brain_Adult_sub.tif
Original image shape: (1200, 1200)
Original image dtype: uint8, max: 255, min: 4
Converted image shape: (1200, 1200, 1)
Image is already in suitable intensity range
Selected channels: [0]
Processed image shape: (1200, 1200, 1)
Initializing Cellpose model (cyto3)...
2025-11-07 11:40:55,430 [INFO] ** TORCH CUDA version installed and working. **
2025-11-07 11:40:55,432 [INFO] >>>> using GPU
2025-11-07 11:40:55,433 [INFO] >> cyto3 << model set to be used
Loading Stereo-seq image: D:/paper/newmodel/data/stereo_seq_mouse_brain/Mouse_brain_Adult_s